<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/SOLCAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas

In [15]:
#@title 🛰️ Hourly Multi-City 2024 (Solar + Wind + Temp)
import requests
import pandas as pd
import time

SOLCAST_API_KEY = "4v9zYe6l-vnTeUdJfwh1kfuM3jGSgKvW"

cities = {
    "Turku": {"lat": 60.5, "lon": 22.3},
    "Copenhagen": {"lat": 55.7, "lon": 12.6},
    "Stockholm": {"lat": 59.3, "lon": 18.1},
    "Oslo": {"lat": 59.9, "lon": 10.7}
}

months = [f"2024-{m:02d}-01" for m in range(1, 13)]
url = "https://api.solcast.com.au/data/historic/radiation_and_weather"

for city, coords in cities.items():
    all_months = []
    print(f"\n🌍 Fetching Full Data for: {city}")

    for start_date in months:
        params = {
            "latitude": coords["lat"],
            "longitude": coords["lon"],
            "api_key": SOLCAST_API_KEY,
            "start": f"{start_date}T00:00:00Z",
            "duration": "P31D",
            "period": "PT1H",
            "format": "json"
        }

        res = requests.get(url, params=params)
        if res.status_code == 200:
            all_months.append(pd.DataFrame(res.json()['estimated_actuals']))
            print(f"  ✅ {start_date} Success", end="\r")
        else:
            print(f"  ❌ {start_date} Failed: {res.status_code}")

        time.sleep(1.2)

    if all_months:
        df = pd.concat(all_months).drop_duplicates(subset=['period_end'])
        df['period_end'] = pd.to_datetime(df['period_end'])

        # Select and rename for clarity
        cols_to_keep = ['period_end', 'ghi', 'dni', 'air_temp', 'wind_speed_10m', 'wind_direction_10m']
        # Check if 100m wind is available in your specific response
        if 'wind_speed_100m' in df.columns: cols_to_keep.append('wind_speed_100m')

        final_df = df[cols_to_keep].copy()
        final_df.to_csv(f"{city}_Solar_Wind_2024.csv", index=False)
        print(f"\n🎉 Finished {city}! Columns: {list(final_df.columns)}")

print("\n🚀 All downloads complete. You now have solar and wind data for all 4 cities.")


🌍 Fetching Full Data for: Turku
  ❌ 2024-01-01 Failed: 402
  ❌ 2024-02-01 Failed: 402
  ❌ 2024-03-01 Failed: 402
  ❌ 2024-04-01 Failed: 402
  ❌ 2024-05-01 Failed: 402
  ❌ 2024-06-01 Failed: 402
  ❌ 2024-07-01 Failed: 402


KeyboardInterrupt: 